# using `Predictor` class

In [ ]:
import boto3
import sagemaker

from torch.utils.data import DataLoader
from torchvision import datasets, transforms

from sagemaker.predictor import Predictor
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

In [ ]:
# set credentials
# but do not hard code credentials as strings. highly not recommended!
ACCESS_KEY_ID = "<your access key id>"
SECRET_ACCESS_KEY = "<your secret access key>"
REGION = "<region in which you deployed the endpoint>"

In [ ]:
# set the endpoint_name defined in the deployment process
endpoint_name = "MultiModelTrial"

# instantiate sessions
boto_session = boto3.Session(
    aws_access_key_id=ACCESS_KEY_ID,
    aws_secret_access_key=SECRET_ACCESS_KEY,
    region_name=REGION
)
session = sagemaker.Session(boto_session=boto_session)

# instantiate predictor
predictor = Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=session,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer()
)

## model_type == 'image'

In [ ]:
# prepare test set
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
dataset = datasets.MNIST('../data', train=False,
                          transform=transform)

# extract only 1 data to test
test_kwargs = {'batch_size': 1}
test_loader = DataLoader(dataset, **test_kwargs)
data, target = next(iter(test_loader))

# set payload input
payload_image = {
    "inputs": test.cpu().detach().numpy().tolist(),
    "model_type": "image"
}

# request prediction
pred_image = predictor.predict(data=payload_image)

## model_type == 'text'

In [ ]:
# set payload input
texts = ["I really like the song", "It's gross! throw it away!"]
payload_text = {
    "inputs": texts,
    "model_type": "text"
}

# request prediction
pred_text = predictor.predict(data=payload_text)

In [ ]:
# see results
print(pred_image)
print(pred_text)

# using invoke_endpoint() method

In [ ]:
import json
import boto3

from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [ ]:
# set credentials
# but do not hard code credentials as strings. highly not recommended!
ACCESS_KEY_ID = "<your access key id>"
SECRET_ACCESS_KEY = "<your secret access key>"
REGION = "<region in which you deployed the endpoint>"

In [ ]:
# set the endpoint_name defined in the deployment process
endpoint_name = "MultiModelTrial"

# instantiate boto3 client
client = boto3.client(
    'sagemaker-runtime',
    REGION,
    aws_access_key_id=ACCESS_KEY_ID,
    aws_secret_access_key=SECRET_ACCESS_KEY
)

## model_type == 'image'

In [ ]:
# prepare test set
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
dataset = datasets.MNIST('../data', train=False,
                          transform=transform)

# extract only 1 data to test
test_kwargs = {'batch_size': 1}
test_loader = DataLoader(dataset, **test_kwargs)
data, target = next(iter(test_loader))

# set payload input
payload_image = {
    "inputs": test.cpu().detach().numpy().tolist(),
    "model_type": "image"
}

# request prediction
response_image = client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=payload_image,
    ContentType='application/json',
    Accept='application/json'
)
result_image = response_image['Body'].read().decode()
pred_image = json.loads(result_image)

## model_type == 'text'

In [ ]:
# set payload input
texts = ["I really like the song", "It's gross! throw it away!"]
payload_text = {
    "inputs": texts,
    "model_type": "text"
}

# request prediction
response_text = client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=payload_text,
    ContentType='application/json',
    Accept='application/json'
)
result_text = response_text['Body'].read().decode()
pred_text = json.loads(result_text)

In [ ]:
# see results
print(pred_image)
print(pred_text)